# Create Customer Summary (Preferences)
- This notebook take about 5 to 10 minutes to execute
- We will create 2 summaries
  - The customer preferences based upon what themes they mention in their reviews
  - A summary of all their reviews for a consolidated overall review  

## Create Summary Prompt and run through LLM

In [ ]:
%%bigquery

-- View the attributes per customer

SELECT *
  FROM `${project_id}.${bigquery_rideshare_llm_enriched_dataset}.customer_attribute`
ORDER BY customer_id,
         customer_attribute_grouping,
         extracted_customer_attribute,
         rank_order
LIMIT 100;

## Aggregate the data for an LLM prompt

In [ ]:
%%bigquery

SELECT customer_id,
       STRING_AGG(extracted_customer_attribute,', ') AS customer_attribute_agg
  FROM `${project_id}.${bigquery_rideshare_llm_enriched_dataset}.customer_attribute`
GROUP BY customer_id
LIMIT 20;

In [ ]:
%%bigquery

-- OPTIONAL: Reset all the fields to null
-- If you need to reset you data back to fresh data run the stored procedure: CALL `${project_id}.${bigquery_rideshare_llm_curated_dataset}.sp_reset_demo`();

/*
UPDATE `${project_id}.${bigquery_rideshare_llm_enriched_dataset}.customer` AS customer
   SET customer_attribute_llm_summary_prompt = NULL,
       llm_summary_customer_attribute_json   = NULL,
       llm_summary_customer_attribute        = NULL
 WHERE TRUE;
*/

In [ ]:
%%bigquery

-- Create the LLM prompt
UPDATE `${project_id}.${bigquery_rideshare_llm_enriched_dataset}.customer` AS customer
   SET customer_attribute_llm_summary_prompt = CONCAT(
'Write a 100 to 600 word summary for the following customer preferences.\n',
'1. The customer\'s name is ', customer.customer_name ,'.\n',
'2. Write the summary in present tense.\n',
'3. Write the summary from the customers prespective.\n',
'4. Do not repeat the same subject in the summary.\n',
'5. Write 3 to 6 sentences.\n',
customer_attribute_agg)
  FROM (SELECT customer_id,
               STRING_AGG(
                CONCAT('Preference: ',
                       extracted_customer_attribute,
                       '.\n')
                       ,'') AS customer_attribute_agg
          FROM `${project_id}.${bigquery_rideshare_llm_enriched_dataset}.customer_attribute`
      GROUP BY customer_id) AS customer_attribute
WHERE customer.customer_id = customer_attribute.customer_id;


In [ ]:
%%bigquery
SELECT customer_attribute_llm_summary_prompt
  FROM `${project_id}.${bigquery_rideshare_llm_enriched_dataset}.customer` AS customer
 WHERE customer_attribute_llm_summary_prompt IS NOT NULL
 LIMIT 10;

## Run the LLM to generate a customer summary

In [ ]:
from google.cloud import bigquery
import pandas as pd

client = bigquery.Client()

In [ ]:
# Process in batches
batch_size = 100

# Set the parameters for more creative
llm_temperature = 1
llm_max_output_tokens = 1024
llm_top_p = 1
llm_top_k = 40

update_sql="""
UPDATE `${project_id}.${bigquery_rideshare_llm_enriched_dataset}.customer` AS customer
  SET llm_summary_customer_attribute_json = child.ml_generate_text_result
  FROM (SELECT *
          FROM ML.GENERATE_TEXT(MODEL`${project_id}.${bigquery_rideshare_llm_enriched_dataset}.cloud_ai_llm_v1`,
              (SELECT customer_id,
                      customer_attribute_llm_summary_prompt AS prompt
                FROM `${project_id}.${bigquery_rideshare_llm_enriched_dataset}.customer`
                WHERE (llm_summary_customer_attribute_json IS NULL
                       OR
                       JSON_VALUE(llm_summary_customer_attribute_json, '$.predictions[0].content') IS NULL
                       )
                  AND include_in_llm_processing = TRUE
                  AND customer_attribute_llm_summary_prompt IS NOT NULL
                LIMIT {batch_size}),
              STRUCT(
                {llm_temperature} AS temperature,
                {llm_max_output_tokens} AS max_output_tokens,
                {llm_top_p} AS top_p,
                {llm_top_k} AS top_k
                ))
  ) AS child
WHERE customer.customer_id = child.customer_id
  """.format(batch_size = batch_size,
             llm_temperature = llm_temperature,
             llm_max_output_tokens = llm_max_output_tokens,
             llm_top_p = llm_top_p,
             llm_top_k = llm_top_k)

print("SQL: {update_sql}".format(update_sql=update_sql))


In [ ]:
# Score while records remain
# score in groups of batch_size records (we can do up to 10,000 at a time)
import time

done = False
displayed_first_sql = False
original_record_count = 0

while done == False:
  # Get the count of records to score
  sql = """
        SELECT COUNT(*) AS cnt
          FROM `${project_id}.${bigquery_rideshare_llm_enriched_dataset}.customer`
         WHERE (llm_summary_customer_attribute_json IS NULL
                OR
                JSON_VALUE(llm_summary_customer_attribute_json, '$.predictions[0].content') IS NULL
                )
           AND include_in_llm_processing = TRUE
           AND customer_attribute_llm_summary_prompt IS NOT NULL;
        """

  df_record_count = client.query(sql).to_dataframe()
  cnt = df_record_count['cnt'].head(1).item()
  if displayed_first_sql == False:
    original_record_count = cnt
    displayed_first_sql = True

  print("Remaining records to process: ", cnt, " out of", original_record_count, " batch_size: ", batch_size)


  if cnt == 0:
    done = True
  else:
    # https://github.com/googleapis/python-bigquery/tree/master/samples
    job_config = bigquery.QueryJobConfig(priority=bigquery.QueryPriority.INTERACTIVE)
    query_job = client.query(update_sql, job_config=job_config)

    # Check on the progress by getting the job's updated state.
    query_job = client.get_job(
        query_job.job_id, location=query_job.location
    )
    print("Job {} is currently in state {}".format(query_job.job_id, query_job.state))

    while query_job.state != "DONE":
      time.sleep(5)
      query_job = client.get_job(
          query_job.job_id, location=query_job.location
          )
      print("Job {} is currently in state {}".format(query_job.job_id, query_job.state))


## Parse the LLM JSON results

In [ ]:
%%bigquery

UPDATE `${project_id}.${bigquery_rideshare_llm_enriched_dataset}.customer` AS customer
   SET llm_summary_customer_attribute = JSON_VALUE(llm_summary_customer_attribute_json, '$.predictions[0].content')
 WHERE llm_summary_customer_attribute_json IS NOT NULL
   AND llm_summary_customer_attribute IS NULL;

In [ ]:
%%bigquery

SELECT customer_id, customer_attribute_llm_summary_prompt, llm_summary_customer_attribute_json, llm_summary_customer_attribute
  FROM `${project_id}.${bigquery_rideshare_llm_enriched_dataset}.customer`
 WHERE llm_summary_customer_attribute_json IS NOT NULL
   AND llm_summary_customer_attribute IS NOT NULL
LIMIT 20;

# Create Customer Summary (Summary of all Reviews)

We will create 2 summaries
1. The customer preferences based upon what themes they mention in their reviews
2. A summary of all their reviews, so we understand their mindset

Customer Summary:
  - Summarize all the customer reviews

## Create Summary Prompt and run through LLM

In [ ]:
%%bigquery

-- OPTIONAL: Reset all the fields to null
-- If you need to reset you data back to fresh data run the stored procedure: CALL `${project_id}.${bigquery_rideshare_llm_curated_dataset}.sp_reset_demo`();

/*
UPDATE `${project_id}.${bigquery_rideshare_llm_enriched_dataset}.customer`
   SET customer_review_summary_llm_summary_prompt = NULL,
       llm_summary_customer_review_summary_json   = NULL,
       llm_summary_customer_review_summary        = NULL
 WHERE TRUE;
*/

In [ ]:
%%bigquery

-- Create the LLM prompt
UPDATE `${project_id}.${bigquery_rideshare_llm_enriched_dataset}.customer` AS customer
   SET customer_review_summary_llm_summary_prompt =
       CONCAT('Write a 100 to 600 word summary for the following customer reviews.\n',
              '1. The reviews are written by ', customer.customer_name, '.\n',
              '2. Write the summary in present tense.\n',
              '3. Do not repeat the same subject in the summary.\n',
              '4. The reviews are for different drivers.\n',
              '5. The reviews are a single rideshare company.\n',
              '6. The drivers all work for the rideshare company.\n',
              '7. Write 3 to 6 sentences.\n',
               customer_review_agg)
  FROM (SELECT customer_id,
               STRING_AGG(CONCAT('Review: ',customer_review_text,'\n'),'') AS customer_review_agg
          FROM `${project_id}.${bigquery_rideshare_llm_enriched_dataset}.customer_review`
      GROUP BY customer_id) AS customer_review
WHERE customer.customer_id = customer_review.customer_id;


## Run the LLM to generate a customer summary

In [ ]:
# Process in batches
batch_size = 100

# Set the parameters for a more creative response
llm_temperature = 1
llm_max_output_tokens = 1024
llm_top_p = 1
llm_top_k = 40

update_sql="""
UPDATE `${project_id}.${bigquery_rideshare_llm_enriched_dataset}.customer` AS customer
  SET llm_summary_customer_review_summary_json = child.ml_generate_text_result
  FROM (SELECT *
          FROM ML.GENERATE_TEXT(MODEL`${project_id}.${bigquery_rideshare_llm_enriched_dataset}.cloud_ai_llm_v1`,
              (SELECT customer_id,
                      customer_review_summary_llm_summary_prompt AS prompt
                FROM `${project_id}.${bigquery_rideshare_llm_enriched_dataset}.customer`
                WHERE include_in_llm_processing = TRUE
                  AND customer_review_summary_llm_summary_prompt IS NOT NULL
                LIMIT {batch_size}),
              STRUCT(
                {llm_temperature} AS temperature,
                {llm_max_output_tokens} AS max_output_tokens,
                {llm_top_p} AS top_p,
                {llm_top_k} AS top_k
                ))
  ) AS child
WHERE customer.customer_id = child.customer_id
  """.format(batch_size = batch_size,
             llm_temperature = llm_temperature,
             llm_max_output_tokens = llm_max_output_tokens,
             llm_top_p = llm_top_p,
             llm_top_k = llm_top_k)

print("SQL: {update_sql}".format(update_sql=update_sql))


In [ ]:
# Score while records remain
# score in groups of batch_size records (we can do up to 10,000 at a time)
import time

done = False
displayed_first_sql = False
original_record_count = 0

while done == False:
  # Get the count of records to score
  sql = """
        SELECT COUNT(*) AS cnt
          FROM `${project_id}.${bigquery_rideshare_llm_enriched_dataset}.customer`
         WHERE (llm_summary_customer_review_summary_json IS NULL
                OR
                JSON_VALUE(llm_summary_customer_review_summary_json, '$.predictions[0].content') IS NULL
                )
           AND include_in_llm_processing = TRUE
           AND customer_review_summary_llm_summary_prompt IS NOT NULL;
        """

  df_record_count = client.query(sql).to_dataframe()
  cnt = df_record_count['cnt'].head(1).item()
  if displayed_first_sql == False:
    original_record_count = cnt
    displayed_first_sql = True

  print("Remaining records to process: ", cnt, " out of", original_record_count, " batch_size: ", batch_size)

  if cnt == 0:
    done = True
  else:
    # https://github.com/googleapis/python-bigquery/tree/master/samples
    job_config = bigquery.QueryJobConfig(priority=bigquery.QueryPriority.INTERACTIVE)
    query_job = client.query(update_sql, job_config=job_config)

    # Check on the progress by getting the job's updated state.
    query_job = client.get_job(
        query_job.job_id, location=query_job.location
    )
    print("Job {} is currently in state {}".format(query_job.job_id, query_job.state))

    while query_job.state != "DONE":
      time.sleep(5)
      query_job = client.get_job(
          query_job.job_id, location=query_job.location
          )
      print("Job {} is currently in state {}".format(query_job.job_id, query_job.state))


## Parse the LLM JSON results

In [ ]:
%%bigquery

UPDATE `${project_id}.${bigquery_rideshare_llm_enriched_dataset}.customer` customer
   SET llm_summary_customer_review_summary = JSON_VALUE(llm_summary_customer_review_summary_json, '$.predictions[0].content')
 WHERE llm_summary_customer_review_summary_json IS NOT NULL
   AND llm_summary_customer_review_summary IS NULL;


In [ ]:
%%bigquery

SELECT customer_id, customer_review_summary_llm_summary_prompt, llm_summary_customer_review_summary_json, llm_summary_customer_review_summary
  FROM `${project_id}.${bigquery_rideshare_llm_enriched_dataset}.customer`
 WHERE llm_summary_customer_review_summary_json IS NOT NULL
   AND llm_summary_customer_review_summary IS NOT NULL
LIMIT 20;
